# Replaying experiments

In this tutorial, we will demonstrate how to reconstruct and visualize previous experiments from stored data. 

Let's import the relevant classes :

In [ ]:
%load_ext param.ipython
import panel as pn

import larvaworld
from larvaworld.lib import reg, util

# Import the simulation class
from larvaworld.lib.sim import ReplayRun

# Import the configuration classes
from larvaworld.lib.reg.generators import ReplayConf, ReplayConfUnit, ReplayConfGroup

larvaworld.VERBOSE = 1

## The Replay Configuration class

The configuration parameters passed to the simulation class are managed by three classes :

- ReplayConfGroup : group-level parameters
- ReplayConfUnit : agent-level parameters
- ReplayConf : all parameters (including the above)

In [ ]:
# Show the attributes of the ReplayConfGroup class
%params ReplayConfGroup

# Show the attributes of the ReplayConfGroup class as a nested dictionary
ReplayConfGroup.param

In [ ]:
# Show the attributes of the ReplayConfUnit class
%params ReplayConfUnit

# Show the attributes of the ReplayConfUnit class as a nested dictionary
ReplayConfUnit.param

In [ ]:
# Show the attributes of the ReplayConf class
%params ReplayConf

# Show the attributes of the ReplayConf class as a nested dictionary
ReplayConf.param

## Replay examples

Now we will specify the dataset to be reconstructed by its unique ID.

It is also possible to locate it by the directory where it is stored 

In [ ]:
reg.conf.Ref.confIDs

In [ ]:
refID = reg.default_refID
# refID = 'Rehydration/AttP2.Deprived'
# refID = 'exploration.dish'
# refID = reg.default_refID
# refID = 'naive_locomotion.20controls'
# refID = 'exploration.150controls'
d = reg.loadRef(refID)

We will specify a number of configuration sets as dictionaries :

In [ ]:
replay_confs = {
    "normal" : {"time_range": (0, 60)},
    "dispersal" : {"transposition": "origin"},
    "fixed_point" : {
                    "agent_ids": [0],
                    "close_view": True,
                    "fix_point": 6,
                    "time_range": (80, 100),
        },
    "fixed_segment" :{
            "agent_ids": [0],
            "close_view": True,
            "fix_point": 6,
            "fix_segment": "rear",
            "time_range": (100, 130),
        },
    "fixed_overlap" : {
            "agent_ids": [0],
            "close_view": True,
            "fix_point": 6,
            "fix_segment": "front",
            "overlap_mode": True,
        },
    "2segs" : {"draw_Nsegs": 2, "time_range": (80, 100)},
    "all_segs" : {"draw_Nsegs": 11, "time_range": (80, 100)},
}

In [ ]:
# This needs debugging
import numpy as np
d.step_data = d.align_trajectories(transposition="origin", replace=True)
xy_max = 2 * np.max(d.step_data[util.nam.xy(d.c.point)].dropna().abs().values.flatten())

p=ReplayConf(**replay_confs["dispersal"]).nestedConf
dd,bg=d.smaller_dataset(p=p)


In [ ]:
# A method that runs the replay simulation
def run_replay(mode):
    p=ReplayConf(refID=refID, **replay_confs[mode]).nestedConf
    rep = ReplayRun(
        parameters=p,
        id=f"{refID}_replay_{mode}",
        dir=f"./media/{mode}",
        screen_kws={"vis_mode": "video", "show_display": True},
    )
    _ = rep.run()

In [ ]:
# Run a normal replay of the dataset
run_replay("normal")

In [ ]:
# Run a reconstructed dispersal experiment where trajectories have benn transposed to the origin
run_replay("dispersal")

In [ ]:
# Substitute the larva body contour by a bisegmental body
run_replay("2segs")

In [ ]:
# ... or by a body with all segments, making use of all the midline points available
run_replay("all_segs")

In [ ]:
# Now let's examine a single individual. Fixate a midline point of the larva body to the arena center
run_replay("fixed_point")

In [ ]:
# Now fixate a midline segment along the y axis
run_replay("fixed_segment")

In [ ]:
# And collapse the entire video to a single image to visualize the flexibility of each segment
run_replay("fixed_overlap")